In [ ]:
# 지도 학습 | sklearn 라이브러리 이용 =================================================
# 분석 순서 : tr/te 분할  ->  y_tr/X_tr, y_te/X_te 분할  ->  X_tr, X_te 정규화  ->  fit()
# loop 내에서 독립변수 변경시  : tr/te 분할(loop 밖) > y_tr/X_tr, y_te/X_te 분리 (loop 내) -> fit() 


# train, test 데이터 분리 ----------------------------------------------------------
# sklearn은 fit_intercept=True가 default라 상수항 추가 불필요
# stratify='col1' 지정시 해당 컬럼의 비율을 유지하도록 층화 추출
from sklearn.model_selection import train_test_split

tr, te = train_test_split(df, train_size=0.8, random_state=123)

y_tr = tr['col1']
X_tr = tr[['col2','col3']]   # X는 컬럼이 하나라도 반드시 2차원 데프 형태로 지정!!!
y_te = te['col1']
X_te = te[['col2','col3']]   # X는 컬럼이 하나라도 반드시 2차원 데프 형태로 지정!!!



# 정규화/표준화 ---------------------------------------------------------------------
# fitting한 instance 재사용 필요시 > scaler 인스턴스 생성 필요  [★★★★★]
# tr/te 분리 후 표준화시, tr로 fit() 후에 tr과 te를 transform() 해야 함!!!
from sklearn.preprocessing  import MinMaxScaler, StandardScaler

sc    = MinMaxScaler().fit(X_tr)
X_trs = pd.DataFrame(sc.transform(X_tr), columns= X_tr.columns)   # 어레이 -> 데프로 반환
X_tes = pd.DataFrame(sc.transform(X_te), columns= X_tr.columns)   # 어레이 -> 데프로 반환


# fitting을 1번만 진행할 경우 > scaler 인스턴스 생성 불필요
sc = MinMaxScaler().fit_transform(df)      # 어레이 반환
sc = StandardScaler().fit_transform(df)    # 어레이 반환
df[['col1','col2']] = sc                   # 어레이를 기존 데프 컬럼에 입력 가능



# VIF 구하기 [★★★★★] ----------------------------------------------------------------
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

X_tr['const'] = 1            # 상수항 추가 조건이 있을 경우 필수!!! (statsmodels 함수라...)
vifs   = [[vif(X_tr, i), X_tr.columns[i]] for i in range(X_tr.shape[1])] # [VIF, 컬럼명] 리스트로 반환
vif_df = pd.DataFrame(vifs, columns = ['VIF', 'VARIABLE'])               # vifs 리스트를 데프로 변환
sorted(vifs, reverse=True)[0]                                            # VIF가 최대인 값과 컬럼명 반환


# model 생성, 학습 (문제에서 주어지는 입력 옵션 반드시 확인!!!) ---------------------
from sklearn.linear_model    import LinearRegression,      LogisticRegression
from sklearn.tree            import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble        import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors       import KNeighborsRegressor,   KNeighborsClassifier
from sklearn.naive_bayes     import GaussianNB

model =       LinearRegression().fit(X_tr, y_tr)  # 입력값 순서(X,y)에 주의
model =     LogisticRegression().fit(X_tr, y_tr)
model =    KNeighborsRegressor().fit(X_tr, y_tr)  # n_neighbors=
model =   KNeighborsClassifier().fit(X_tr, y_tr)  # 맨해튼 거리 : p=1, 거리 역순 : weights='distance'
model =  DecisionTreeRegressor().fit(X_tr, y_tr)  # max_depth=, random_state=
model = DecisionTreeClassifier().fit(X_tr, y_tr)  
model =  RandomForestRegressor().fit(X_tr, y_tr)  # n_estimators=트리수, min_samples_leaf=분기멈춤 최소 샘플
model = RandomForestClassifier().fit(X_tr, y_tr) 
model =             GaussianNB().fit(X_tr, y_tr)


# model 결과 속성값 확인 --------------------------------------------------------------
model.coef_                # 기울기 (regressor)
odds = np.exp(model.coef_) # odds비 계산

model.intercept_           # 절편 (regressor)
model.feature_importances_ # feature 중요도 (classifier)


# predict -----------------------------------------------------------------------------
# 학습 데이터가 표준화된 경우, 회귀값 예측은 반드시 표준화 전 데이터 사용!!! (분류는 관계 없음)
y_pr       = model.predict(X_te)        # classifier : 1, 0 값 | regressor : 예측 값
y_pr_proba = model.predict_proba(X_te)  # classifier : [0 확률 값, 1 확률 값] 반환


# Regressor 성능평가 ------------------------------------------------------------------
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

mse  = ((y_te-y_pr)**2).mean()             # 계산식 결과는 함수값과 반드시 비교!!! (y_te는 시리즈) 

y_pr = np.array([y_pr]*y_te.shape[0])      # y_pr 이 1개인 경우, 함수 이용시 y_te 행 갯수만큼 복사 필요
mse  = mean_squared_error(y_te, y_pr)      
rmse = mse**0.5                            # 또는 rmse = np.sqrt(mse)

mae  = mean_absolute_error(y_te, y_pr)    
mape = mean_absolute_percentage_error(y_te, y_pr)  # *100을 해야 퍼센트값 임 주의!!!

r2   = r2_score(y_te, y_pr)                # 참값, 예측값 입력 순서 중요!!!


# Classifier 성능평가 -----------------------------------------------------------------
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

accuracy  =  accuracy_score(y_te, y_pr)    # 0~1 | 0~100범위 확인하고 답을 적자!!!
roc       =   roc_auc_score(y_te, y_pr)
f1        =        f1_score(y_te, y_pr)
precision = precision_score(y_te, y_pr)
recall    =    recall_score(y_te, y_pr)

In [ ]:
# 비지도 학습 | skearn K-Means 라이브러리 이용 ===========================================


# 정규화/표준화 ---------------------------------------------------------------------
# fitting을 1번만 진행할 경우 > scaler 인스턴스 생성 불필요
from sklearn.preprocessing  import MinMaxScaler, StandardScaler

sc = MinMaxScaler().fit_transform(df)      # 어레이 반환
sc = StandardScaler().fit_transform(df)    # 어레이 반환
df[['col1','col2']] = sc                   # 어레이를 기존 데프 컬럼에 입력 가능


# model 생성, 학습 (문제에서 주어지는 입력 옵션 반드시 확인!!!) -------------------------
# n_clusters=클러스터 갯수, n_init= 초기 중심점 랜덤 선택 횟수, max_iter=반복횟수, random_state=123
# min_samples_leaf= 가지에 남는 최소의 샘플 갯수 [★★★★★]
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3, n_init=50, random_state=123).fit(df)  # 어레이|데프 입력 가능


# model 결과 속성값 확인 ----------------------------------------------------------------
# 군집 결과를 이용해 추가 계산시 반드시 표준화 전의 데이터 이용!!!!!
# 군집 결과를 원본 df에 컬럼으로 추가시 후속 분석 작업에 영향을 미치니 주의!!!
pd.Series(model.labels_).value_counts()                 # 군집별 요소 갯수 반환 (시리즈 변환 필수)
pd.Series(model.labels_).value_counts(normalize=True)   # 군집별 상대도수 반환  (시리즈 변환 필수)
pd.Series(model.labels_).value_counts().max()           # 군집별 요소 갯수 중 최대값 반환


# 군집 중심점간의 거리 계산 [★★★★★] -----------------------------------------------------
from scipy.spatial.distance import pdist      # pairwise distance (두점 간 거리 계산)

model.cluster_centers_             # 중심점 값
pdist(model.cluster_centers_)      # 중심점 간 거리 (metric='euclidean'|'cityblock'(맨하탄 거리))


# 성능 평가 -----------------------------------------------------------------------------
from sklearn.metrics import silhouette_score, silhouette_samples # samples를 평균한 값이 score

silhouette_score(df, model.labels_)       # (fit용 데이터, label 값) 입력값, 순서 주의!!!
silhouette_samples(df, model.labels_)     # (fit용 데이터, label 값) 입력값, 순서 주의!!!


# (실루엣 계수, k값) 리스트 생성 후 n번째 값 반환 [★★★★★]
l1 =[]
for k in range(2, 10):
    model = KMeans(n_clusters=k, random_state = 123).fit(df)
    sc = silhouette_score(df, model.labels_)
    l1 += [(sc, k)]                # (실루엣 값, k값) 튜플의 리스트 (정렬 기준 값 먼저 입력)
r1 = sorted(l1, reverse=True)[0]   # 실루엣 값 기준 내림차순 정렬 후, n번째 값 선택 (최대값은 [0])

In [ ]:
# 지도 학습 | statsmodels 라이브러리 이용 ================================================
# sklearn만으로 모든 것이 가능하기 때문에 굳이 statsmodels를 이용할 필요는 없음
# sklearn은 fit_intercept=True가 default라 상수항 추가 불필요

# train, test 데이터 분리 -------------------------------------------------------------
# stratify='col1' 지정시 해당 컬럼의 비율을 유지하도록 층화 추출
from sklearn.model_selection import train_test_split

tr, te = train_test_split(df, train_size=0.8, random_state=123)

y_tr = tr['col1']
X_tr = tr[['col2','col3']]   # X는 컬럼이 하나라도 반드시 2차원 데프 형태로 지정!!!
y_te = te['col1']
X_te = te[['col2','col3']]   # X는 컬럼이 하나라도 반드시 2차원 데프 형태로 지정!!!


# VIF 구하기 [★★★★★] ----------------------------------------------------------------
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

X_tr['const'] = 1            # 상수항 추가 조건이 있을 경우 필수!!! (statsmodels 함수라...)
vifs   = [[vif(X_tr, i), X_tr.columns[i]] for i in range(X_tr.shape[1])] # [VIF, 컬럼명] 리스트로 반환
vif_df = pd.DataFrame(vifs, columns = ['VIF', 'VARIABLE'])               # vifs 리스트를 데프로 변환
sorted(vifs, reverse=True)[0]                                            # VIF가 최대인 값과 컬럼명 반환


# model 생성, 학습 (문제에서 주어지는 입력 옵션 반드시 확인!!!) ----------------------
from statsmodels.api import OLS, Logit

model =   OLS(y_tr, X_tr).fit()   # 입력 위치와 입력값 순서(y, X)에 주의
model = Logit(y_tr, X_tr).fit()


# model 결과 속성값 확인 --------------------------------------------------------------
model.params     # 절편, 기울기 값 (regressor)
model.rsquared   # R2 (regressor)
model.summary()  # R2, Adj-R2, 회귀계수 등 (regressor)


# predict -----------------------------------------------------------------------------
y_pr = model.predict(X_te)      # classifier : 확률 값 반환 (주의!!), regressor: 예측 값
y_pr = (y_pred>0.5).astype(int) # classifier : 확률값을 1,0 값으로 변환


# Regressor 성능평가 ------------------------------------------------------------------
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

mse  = ((y_te-y_pr)**2).mean()             # 계산식 결과는 함수값과 반드시 비교!!! (y_te는 시리즈) 

y_pr = np.array([y_pr]*y_te.shape[0])      # y_pr 이 1개인 경우, 함수 이용시 y_te 행 갯수만큼 복사 필요
mse  = mean_squared_error(y_te, y_pr)      
rmse = mse**0.5                            # 또는 rmse = np.sqrt(mse)

mae  = mean_absolute_error(y_te, y_pr)    
mape = mean_absolute_percentage_error(y_te, y_pr)  # *100을 해야 퍼센트값 임 주의!!!

r2   = r2_score(y_te, y_pr)                # 참값, 예측값 입력 순서 중요!!!



# Classifier 성능평가 -----------------------------------------------------------------
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

accuracy  =  accuracy_score(y_te, y_pr)    # 0~1 | 0~100범위 확인하고 답을 적자!!!
roc       =   roc_auc_score(y_te, y_pr)
f1        =        f1_score(y_te, y_pr)
precision = precision_score(y_te, y_pr)
recall    =    recall_score(y_te, y_pr)


# Loop 계산시 성능평가 결과를 리스트로 반환하는 방법
l1 = []
for idx in range(5):
    # 모델 생성 > 훈련 > 예측 과정 수행 #
    as1 = accuracy_score(y_te, y_pr)
    l1 += [(as1, idx)]
r1 = sorted(l1, reverse=True)[0]   # accuray score 기준 내림차순 정렬 후, n번째 값 선택 (최대값은 [0])